In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
chromedriver = 'D:/20220103_lab/14.python/data/chrome/chromedriver'
driver = webdriver.Chrome(chromedriver) 

# 포스팅 작성 당시 크롬 버젼 : 92

In [ ]:
df = pd.DataFrame

In [ ]:
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''

In [ ]:
# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

In [ ]:
# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [ ]:
# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
Python


In [ ]:
from selenium import webdriver
import time

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

In [ ]:
# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [ ]:
# 여러 페이지(999)에서 반복하기
for i in range(999):
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")
    

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

            # 가게 주소 데이터 수집 // span.section-result-location

            addr = s.find_element_by_css_selector("span.section-result-location").text

            print(title, "/", score, "/", addr)
            
    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
stores

In [ ]:
# 크롬창 닫기
driver.close()
# 출처: https://datalabbit.tistory.com/45 [간토끼 DataMining Lab:티스토리]

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
driver.get("https://www.google.com/maps/")

In [ ]:
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [ ]:
for i in range(999):
    time.sleep(3)
    stores = driver.find_elements_by_css_selector("QA0Szd")
    for s in stores:
        print(s)
        title = s.find_element_by_css_selector("h3.section-result-title").text
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"
            
            addr = s.find_element_by_css_selector("span.section-result-location").text
            
            print(title, "/", score, "/", addr)
            
    try:
        nextpage = driver.find_element_by_css_selebctor("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
s

In [ ]:
title

In [ ]:
driver.close()

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
driver.get("https://www.google.com/maps/")

In [ ]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [ ]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [ ]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]')

In [ ]:
target.text

In [ ]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [ ]:
driver.execute_script("window.scrollTo(0, 1000)") 

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
last_height

In [ ]:
SCROLL_PAUSE_SEC = 1
while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# ActionChains 를 사용하기 위해서.
from selenium.webdriver import ActionChains

# id가 something 인 element 를 찾음
some_tag = driver.find_element(By.ID, 'pane')

# somthing element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(some_tag).perform()


In [ ]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

In [ ]:
for divtag in target.find_elements(By.XPATH, './div'):
    if divtag.text != "":
        print("--------------------------")
        atag = divtag.find_element(By.XPATH, './div/a[1]')
        subdiv = divtag.find_element(By.XPATH, "./div/div[2]")
        name = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
        star = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
        addr = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]")

        print(name.text)
        print(addr.text)
        print(atag.get_attribute('href').split('!')[5][2:])
        print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
        print(star.text)
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
driver.get("https://map.kakao.com/")

In [ ]:
search_input_tag = driver.find_element(By.ID, 'search.keyword.query')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [ ]:
search_button_tag = driver.find_element(By.ID, 'search.keyword.submit')
search_button_tag.click()
time.sleep(5)

In [ ]:
place_more = driver.find_element(By.ID, 'info.search.place.more')
place_more.click()
time.sleep(5)

In [ ]:
for i in range(999):
    nextpage = driver.find_element(By.ID, "info.search.page.no{}".format((i+1)%5))
    nextpage.click()
    print(i)
    if i == 4 :
        nextpage = driver.find_element(By.ID, "info.search.page.next").click()
        time.sleep(1)
        i = 1

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [3]:
search_input_tag = driver.find_element(By.CLASS_NAME, 'mL3xi')
search_input_tag.send_keys('동물병원')
time.sleep(3)


NameError: name 'driver' is not defined

In [4]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)

C:\Users\82103\AppData\Local\Temp/ipykernel_19068/451005478.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [5]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)

In [6]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [ ]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [ ]:
driver.execute_script("document.body.style.zoom='100%'")

#elem.send_keys(Keys.ENTER)

In [7]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[2]')

In [8]:
target.text

'평점\n영업시간\n모든 필터\n아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n지금 영업 중 · 032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n이지동물병원\n4.5(8)\n동물 병원 · 병방동 422\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-549-0077\n경로\n신통방통동물병원\n4.9(10)\n동물 병원 · 작전동 863\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-552-3369\n경로\n작전24동물병원\n4.4(7)\n동물병원 · 작전동 925-1\n지금 영업 중\n경로\n재영동물병원\n3.8(4)\n동물병원 · 계산1동\n지금 영업 중 · 032-542-4647\n경로\n검색결과 1 - 20 표시 중\n지도 이동 시 결과 업데이트'

In [ ]:
panee = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[8]')

class = "m6QErb DxyBCb kA9KIf dS8AEf ecceSd"

In [ ]:
asdasdf = driver.find_element_by_css_selector("div.rHNip.cRLbXd")

In [ ]:
asdasdf.click()

In [ ]:
asdasdf.send_keys(Keys.END)

In [ ]:
akjsvebjkseagvf2 = driver.find_element_by_class_name("rHNip cRLbXd")


In [ ]:
akjsvebjkseagvf2 = driver.find_element_by_css_selector("div.drGLxe")

In [ ]:
akjsvebjkseagvf2.send_keys(Keys.ENTER)

In [ ]:
akjsvebjkseagvf = driver.find_element(By.CLASS_NAME, "tXNTee LCTIRd L6Bbsd VbYL9")

In [ ]:
jksdfkhsbdf = driver.find_element_by_css_selector("div.drGLxe")

In [ ]:
jksdfkhsbdf.click()

In [ ]:
target.text

In [ ]:
target.send_keys(Keys.END)

In [ ]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [ ]:
driver.execute_script('arguments[0].scrollIntoView(true);', target)

In [ ]:
actions = ActionChains(driver) \
    .move_to_element(panee) \
    .click() \
    .send_keys(Keys.END)

actions.perform() 

In [ ]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

divtag -> class -> "m6QErb DxyBCb kA9KIf dS8AEf ecceSd QjC7t"

subdiv -> jstcache="184" 들 모아놓음

name -> class="NrDZNb"

star -> jstcache="83"

In [ ]:
target

In [ ]:
target.text

In [9]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[2]')

In [10]:
lname = []
lstar = []
laddr = []

In [73]:
for divtag in target.find_elements(By.XPATH, "./div[1]"):
    if divtag.text != "":
        for i in lista:
            print("--------------------------")
            atag = divtag.find_element(By.XPATH, "./div[{}]/div[1]/a[1]".format(i))
            subdiv = divtag.find_element(By.XPATH, "./div[{}]".format(i))
            name = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
            star = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
            addr = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]")

            print(name.text)
            print(star.text)
            print(addr.text)
            
#            print(atag.get_attribute('href').split('!')[5][2:])
#            print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
            
            lname.append(name.text)
            lstar.append(star.text)
            laddr.append(addr.text)
            
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

--------------------------
세종다온동물병원
리뷰 없음
동물 병원 · 세종 소담동 37-3
지금 영업 중 · 044-865-7579
--------------------------


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[5]/div[1]/a[1]"}
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x006CB8F3+2406643]
	Ordinal0 [0x0065AF31+1945393]
	Ordinal0 [0x0054C748+837448]
	Ordinal0 [0x005792E0+1020640]
	Ordinal0 [0x0057957B+1021307]
	Ordinal0 [0x0056FAE1+981729]
	Ordinal0 [0x005942C4+1131204]
	Ordinal0 [0x0056FA64+981604]
	Ordinal0 [0x00594494+1131668]
	Ordinal0 [0x005A4682+1197698]
	Ordinal0 [0x00594096+1130646]
	Ordinal0 [0x0056E636+976438]
	Ordinal0 [0x0056F546+980294]
	GetHandleVerifier [0x00939612+2498066]
	GetHandleVerifier [0x0092C920+2445600]
	GetHandleVerifier [0x00764F2A+579370]
	GetHandleVerifier [0x00763D36+574774]
	Ordinal0 [0x00661C0B+1973259]
	Ordinal0 [0x00666688+1992328]
	Ordinal0 [0x00666775+1992565]
	Ordinal0 [0x0066F8D1+2029777]
	BaseThreadInitThunk [0x7623FA29+25]
	RtlGetAppContainerNamedObjectPath [0x775A7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77

In [12]:
lista = []
for x in list(range(50)):
    if x % 2 :
        lista.append(x)

In [15]:
lista

[3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49]

In [14]:
lista.remove(1)

In [ ]:
print(len(lname))
lname260 = lname

In [ ]:
len(lname260)

In [ ]:
print(len(lstar))
lstar260 = lstar

In [ ]:
print(len(laddr))
laddr260 = laddr

In [72]:
lname

['아마존동물병원',
 '계양동물메디컬센터',
 '해림동물병원',
 '한솔동물병원',
 '아프리카 동물의료센터 [구 아프리카동물병원]',
 '다오동물병원',
 '이지동물병원',
 '신통방통동물병원',
 '작전24동물병원',
 '재영동물병원',
 '신영재동물병원',
 '부평종합동물병원',
 '보비동물병원',
 '부평종합동물의료센터',
 '부천준동물병원',
 '세림동물병원',
 '부천 TLC 동물의료센터',
 '포근한동물병원',
 '24시나음동물병원',
 '펫사랑동물병원',
 '24시 해든동물메디컬센터',
 '짱구네동물병원',
 '리치동물병원',
 '지중해동물병원',
 '한사랑동물병원',
 '소나무동물메디컬센터',
 '24시 SKY 동물병원',
 '펫가든동물병원',
 '위더스동물병원',
 '가정동물병원',
 '청라라임동물병원',
 '웰케어동물메디컬센터',
 '청천종합동물병원',
 '그랜드동물병원',
 '위즈펫동물병원',
 '닥터팬더동물병원',
 '동행동물병원',
 '현대동물병원',
 '중동동물병원',
 '동수동물병원',
 '하얀기린동물병원',
 '이레동물병원',
 '온유동물병원',
 '해오름동물병원',
 '삼성동물병원',
 '백마동물병원',
 '24시동물병원',
 'AVC동물메디컬센터',
 '쿨펫동물병원',
 '24시동물병원',
 '계양동물병원',
 '정서동물병원',
 '조재진동물병원',
 '하노바동물병원',
 '청라루비동물병원',
 '검암동물병원',
 '이오동물병원',
 '쿨펫동물병원이마트계양점',
 '다나동물병원',
 '튼튼동물병원',
 '부평모모동물병원',
 '주주종합동물병원',
 '행복한동물병원',
 '도그앤캣동물병원',
 '부평프라임동물병원',
 '24시독스동물병원',
 '비엔동물전문의료센터',
 '아이케어동물병원',
 '브레멘애니멀스',
 '늘푸른동물병원',
 '가좌동물병원',
 '길동물병원',
 'BE동물병원',
 '신정동물병원',
 '펫홈종합동물병원',
 '하나동물병원',
 '청라종합동물병원',
 '서울종합동물병원',
 '아이러브펫동물병원작전점',


In [ ]:
lstar

In [ ]:
laddr

In [74]:
print(len(lname))

282


In [75]:
print(len(lstar))

282


In [76]:
print(len(laddr))

282


In [77]:
asdfa2 = []
for x in laddr:
    asdfa2.append(x.split("\n"))


In [78]:
asdfa2

[['동물병원 · 임학동 67-7', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3'],
 ['동물병원 · 계산동 942-24번지', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975'],
 ['동물병원 · 작전동 864-39', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079'],
 ['동물병원 · 계산2동 918-2', '지금 영업 중 · 032-553-4652'],
 ['동물병원 · 장제로340번길 5 3층', '영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582'],
 ['동물병원 · 계산동 1062', '032-548-0275'],
 ['동물 병원 · 병방동 422', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-549-0077'],
 ['동물 병원 · 작전동 863', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-552-3369'],
 ['동물병원 · 작전동 925-1', '지금 영업 중'],
 ['동물병원 · 계산1동', '지금 영업 중 · 032-542-4647'],
 ['동물병원 · 병방동 426', '032-552-0075'],
 ['동물병원 · 부평동 부평대로 154', '영업 중 ⋅ 오후 8:00에 영업 종료 · 032-511-6836'],
 ['동물병원 · 산곡동 411-5', '032-524-6959'],
 ['동물병원 · 부평대로 138', '지금 영업 중 · 032-511-6836'],
 ['동물병원 · 중3동 1062-1', '지금 영업 중 · 032-322-7595'],
 ['동물병원 · 산곡동 143-2', '지금 영업 중 · 032-503-0075'],
 ['동물병원 · 중동 길주로 281 TLC 동물의료센터 1층', '24시간 영업 · 032-323-2475'],
 ['동물병원 · 중동 1168', '영업 중 ⋅ 오전 12:00에 영업 종료 · 032-231-0075'],
 ['동물 병원 · 중동', '지금 영업 중 · 0

In [82]:
import pandas as pd

In [98]:
laddr = pd.DataFrame(asdfa2)[0]

In [84]:
df_addr3 = pd.DataFrame(df_addr2)

NameError: name 'df_addr2' is not defined

In [86]:
pd.DataFrame(asdfa2)[1]

0            영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3
1                   영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2                   영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3                                 지금 영업 중 · 032-553-4652
4                  영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
                             ...                        
277                                         031-709-5500
278                                          02-522-4722
279    곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280    곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...
281                               지금 영업 중 · 044-865-7579
Name: 1, Length: 282, dtype: object

In [87]:
df_openphone = pd.DataFrame(pd.DataFrame(asdfa2)[1])

In [88]:
df_openphone

,1
0,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3
1,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3,지금 영업 중 · 032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
277,031-709-5500
278,02-522-4722
279,곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280,곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...


In [89]:
df_name = pd.DataFrame(lname)

In [90]:
df_star = pd.DataFrame(lstar)

In [99]:
df_addr = pd.DataFrame(laddr)

In [92]:
df_name.columns = ["병원이름"]

In [93]:
df_star.columns = ["리뷰"]

In [100]:
df_addr.columns = ["주소"]

In [101]:
df_name.head(5)

,병원이름
0,아마존동물병원
1,계양동물메디컬센터
2,해림동물병원
3,한솔동물병원
4,아프리카 동물의료센터 [구 아프리카동물병원]


In [102]:
df_star.head(5)

,리뷰
0,4.6(25)
1,4.3(50)
2,4.8(100)
3,4.3(13)
4,4.4(89)


In [103]:
df_addr.head(5)

,주소
0,동물병원 · 임학동 67-7
1,동물병원 · 계산동 942-24번지
2,동물병원 · 작전동 864-39
3,동물병원 · 계산2동 918-2
4,동물병원 · 장제로340번길 5 3층


In [104]:
df_addr2.head(5)

NameError: name 'df_addr2' is not defined

In [105]:
df_addr3.head(5)

NameError: name 'df_addr3' is not defined

In [106]:
df_openphone.columns = ["오픈마감시간, 전화번호"]

In [107]:
df_openphone

,"오픈마감시간, 전화번호"
0,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3
1,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3,지금 영업 중 · 032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
277,031-709-5500
278,02-522-4722
279,곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280,곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...


In [108]:
df_openphone["오픈마감시간, 전화번호"].isnull().sum()

4

In [109]:
len(df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].isnull()])

4

In [110]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].isnull()] = 0

In [111]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == 0] = " "

In [112]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == " "] = "정보가 없습니다."

In [113]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == "정보가 없습니다."]

,"오픈마감시간, 전화번호"
34,정보가 없습니다.
57,정보가 없습니다.
196,정보가 없습니다.
275,정보가 없습니다.


11,12,37,39,49,71,79,89,184,202,230,236,255,278

In [114]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains('ext')]

,"오픈마감시간, 전화번호"
0,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3
82,영업 중 ⋅ 오후 7:00에 영업 종료 · 02-2645-1475 ext. 1074


In [115]:
df_openphone_ext = df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains("ext")]

In [116]:
len(df_openphone_ext["오픈마감시간, 전화번호"][0])

43

In [117]:
df_openphone_ext["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:51]

C:\Users\82103\AppData\Local\Temp/ipykernel_19068/1080749331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_openphone_ext["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:51]


In [120]:
len(df_openphone_ext["오픈마감시간, 전화번호"][82])

46

In [121]:
df_openphone_ext["오픈마감시간, 전화번호"][82] = df_openphone_ext["오픈마감시간, 전화번호"][82][:39]

C:\Users\82103\AppData\Local\Temp/ipykernel_19068/1366251628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_openphone_ext["오픈마감시간, 전화번호"][82] = df_openphone_ext["오픈마감시간, 전화번호"][82][:39]


In [139]:
df_openphone_ext["오픈마감시간, 전화번호"][0][:36]

'영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502'

In [140]:
df_openphone["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:36]

In [128]:
df_openphone["오픈마감시간, 전화번호"][82] = df_openphone_ext["오픈마감시간, 전화번호"][82][:36]

In [141]:
df_openphone["오픈마감시간, 전화번호"][82]

'영업 중 ⋅ 오후 7:00에 영업 종료 · 02-2645-1475'

In [142]:
df_openphone["오픈마감시간, 전화번호"][0][-12:]

'032-555-7502'

In [143]:
df_openphone["오픈마감시간, 전화번호"][58][-12:]

'032-323-2737'

In [144]:
df_openphone["오픈마감시간, 전화번호"][277][-12:]

'031-709-5500'

In [145]:
df_openphone["오픈마감시간, 전화번호"]

0                   영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502
1                   영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2                   영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3                                 지금 영업 중 · 032-553-4652
4                  영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
                             ...                        
277                                         031-709-5500
278                                          02-522-4722
279    곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280    곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...
281                               지금 영업 중 · 044-865-7579
Name: 오픈마감시간, 전화번호, Length: 282, dtype: object

In [146]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains("")]

,"오픈마감시간, 전화번호"
0,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502
1,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3,지금 영업 중 · 032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
277,031-709-5500
278,02-522-4722
279,곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280,곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...


In [ ]:
df_animalhospital = pd.concat([df_name, df_star, df_addr3, df_openphone], axis=1)

In [147]:
df_animalhospital = pd.concat([df_name, df_star, df_addr, df_openphone], axis=1)

In [148]:
df_animalhospital

,병원이름,리뷰,주소,"오픈마감시간, 전화번호"
0,아마존동물병원,4.6(25),동물병원 · 임학동 67-7,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502
1,계양동물메디컬센터,4.3(50),동물병원 · 계산동 942-24번지,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975
2,해림동물병원,4.8(100),동물병원 · 작전동 864-39,영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079
3,한솔동물병원,4.3(13),동물병원 · 계산2동 918-2,지금 영업 중 · 032-553-4652
4,아프리카 동물의료센터 [구 아프리카동물병원],4.4(89),동물병원 · 장제로340번길 5 3층,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...,...,...,...
277,반려동물암센터,리뷰 없음,동물병원 · 황새울로342번길 15 은성빌딩,031-709-5500
278,한국동물병원협회,리뷰 없음,협회 또는 조직 · 서초2동 1358-1,02-522-4722
279,녹십자동물병원,4.3(24),동물병원 · 부여읍 구아리 141-5,곧 영업 종료: 오후 6:00 ⋅ 월 오전 9:00에 영업 시작 · 041-832-...
280,충남대학교 동물병원,4.5(14),동물병원 · 궁동 220번지,곧 영업 종료: 오후 6:00 ⋅ 월 오전 10:00에 영업 시작 · 042-821...


In [149]:
df_animalhospital.columns = ["병원이름", "리뷰", "주소", "오픈마감전화번호"]

In [150]:
df_animalhospital["주소"] = df_animalhospital["주소"].apply(lambda x:x.replace("동물병원 · ", ""))

In [151]:
df_animalhospital["주소"] = df_animalhospital["주소"].apply(lambda x:x.replace("동물 병원 · ", ""))

In [152]:
pd.set_option('display.max_row', 300)

In [153]:
df_animalhospital[["주소"]]

,주소
0,임학동 67-7
1,계산동 942-24번지
2,작전동 864-39
3,계산2동 918-2
4,장제로340번길 5 3층
5,계산동 1062
6,병방동 422
7,작전동 863
8,작전동 925-1
9,계산1동


In [154]:
pd.set_option('display.max_row', 10)

In [155]:
df_animalhospital[["주소"]]

,주소
0,임학동 67-7
1,계산동 942-24번지
2,작전동 864-39
3,계산2동 918-2
4,장제로340번길 5 3층
...,...
277,황새울로342번길 15 은성빌딩
278,협회 또는 조직 · 서초2동 1358-1
279,부여읍 구아리 141-5
280,궁동 220번지


In [157]:
df_animalhospital2 = df_animalhospital[["병원이름", "주소", "리뷰"]]

In [158]:
df_animalhospital2

,병원이름,주소,리뷰
0,아마존동물병원,임학동 67-7,4.6(25)
1,계양동물메디컬센터,계산동 942-24번지,4.3(50)
2,해림동물병원,작전동 864-39,4.8(100)
3,한솔동물병원,계산2동 918-2,4.3(13)
4,아프리카 동물의료센터 [구 아프리카동물병원],장제로340번길 5 3층,4.4(89)
...,...,...,...
277,반려동물암센터,황새울로342번길 15 은성빌딩,리뷰 없음
278,한국동물병원협회,협회 또는 조직 · 서초2동 1358-1,리뷰 없음
279,녹십자동물병원,부여읍 구아리 141-5,4.3(24)
280,충남대학교 동물병원,궁동 220번지,4.5(14)


In [160]:
df_animalhospital2.to_csv('animalhospital.csv')

In [163]:
df_animalhospital2.duplicated().sum()

0